# Sound Analysis for Summer Ventures - Interactive Figures

This version of the Sound Analysis notebook produces interactive figures using *Plotly*. These interactive figures are good for exploring graphs. They can be exported to Javascript-powered interactive HTML files that can be opened in any browser or embedded in a webpage. The interactive figures are **not** good for embedding in a PowerPoint presentation or using as figures in a paper. Use the *Static Figures* verion of this notebook to produce figures to embed in presentations or papers.

This Jupyter notebook will allow you to do analysis of sound recordings and make graphs using the Plotly modules. Get started by following the directions before each cell. Change the parameters as needed, and execute the commands in the cell by entering `Shift-Enter` or `Shift-Return` with your the cell selected or with your cursor in the cell.

Remember that you the order that you enter cells matters, not the order in which they appear in the notebook.

## Load Python Modules

Enter the cell below to load the Python modules you will need. Add any additional modules you need for your specific calculations.

In [0]:
import numpy as np # Numerical analysis package
import scipy.signal as signal # Signal processing module
import plotly.graph_objects as go # Interactive graph module
from plotly.subplots import make_subplots # Produce figure with multiple subplots.
from SVSound import wavefile # Module to read WAV files

## Extra configuration for *Juno Connect* app
If you are using the app *Juno Connect* on an *iPad* or *iPhone*, you must enter the commands below to configure *Plotly* to produce figures that your notebook can display. If you are using a browser or other interface, **do not** enter the commands unless your graphs do not display otherwise. (Feel free to delete them if not needed.)

In [0]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Read WAV File(s)

You will use `wavefile.read` to read the information and data from sound recordings in the WAV format. This function was written to read WAV files written by many different devices, including the smartphone apps you may use.

The `wavefile.read` function returns the file parameters and the data so you can use them for your analysis. As with most Python programs, enter `?wavefile` and `?wavefile.read` to get more information about them.

Enter the cell below to read the file `Beats.wav`, which has been loaded into this project. If you want to read a different WAV file, upload it to your project folder and change the `"Beats.wav"` in the command below to your filename (in quotes). (As always, feel free to change variable names `beats_info` and `beats_wave` to something that works for you.)

In [0]:
beats_info, beats_wave = wavefile.read("Beats.wav")

Now we are going to make variable with the times of each sample in `beats_wave`. We do this by taking a list of all the samples (0, 1, 2, 3, ...) amd dividing by the sampling frequency (because period $T = 1/f$). Do this by entering the cell below.

In [0]:
beats_times = np.arange(beats_info['Nsamples']) / beats_info['fs']

Now we will open a second WAV file, `OrganPipeOpenClosed.wav` so you can have two examples. Enter the cell below to run the same command we did above for this file.

In [0]:
pipe_info, pipe_wave = wavefile.read("OrganPipeOpenClosed.wav")
pipe_times = np.arange(pipe_info['Nsamples']) / pipe_info['fs']

## Plot Waveforms

The cell below generates a plot of the entire `beats_wave` variable vs. the time values stored in `beats_times`. The resulting graph has buttons for zooming, panning, etc. Enter the cell below to beterate the graph.

In [0]:
fig = go.Figure(go.Scatter(x=beats_times, y=beats_wave, line={'color': 'blue'}))
fig.update_xaxes(title_text='Time (s)')
fig.update_yaxes(title_text='Relative Pressure')
fig.show()

You can save graphs to an HTML files that are interactive figures you can open in a browser. Provide the exported file name in quotes. Enter the cell below to save the figure to an HTML file in this project directory. (You can download it to use it.)

In [0]:
fig.write_html("Beats Wave Plot.html")

Now plot the second sound `pipe_wave`.

In [0]:
fig = go.Figure(go.Scatter(x=pipe_times, y=pipe_wave, line={'color': 'blue'}))
fig.update_xaxes(title_text='Time (s)')
fig.update_yaxes(title_text='Relative Pressure')
fig.show()

You can save this plot using the same commands as the previous plot. Be sure to change the output filename.

## Power Spectrum

First we will produce a power spectrum of the entire first sound stored in `beats_wave` with parameters stored in `beats_info`. Begin by displaying the sampling frequency.

There is a tradeoff in FFTs and power spectra. You need to take more FFT samples to get a greater frequency resolution, but that requires you to sample a larger total time. If you are interested in frequency resolution, choose a large FFT size. If you want averaging, choose a smaller FFT size.

Now we will produce power spectra of portions of the first sound. First enter the number of FFT points you want to use. In this case, we are going to set the number of FFT points to the same number as the sampling frequency. This will give us a total sampling time of 1 s and a frequency resolution of 1 Hz.

In [0]:
nfft = beats_info["fs"]

Now enter the cell below to calculate the power spectrum of the entire sound.

In [0]:
beats_freq, beats_ps = signal.welch(beats_wave, beats_info['fs'], nperseg=nfft)

Now enter the cell below to graph the power spectrum. Notice that we take the logarithm of the power spectrum and multiply by 10 before plotting. This converts it to decibels.

In [0]:
fig = go.Figure(go.Scatter(x=beats_freq, y=10*np.log10(beats_ps), 
                                   line={'color': 'blue'}))
fig.update_xaxes(title_text='Frequency (Hz)')
fig.update_yaxes(title_text='Power Spectral Density (dB)')
fig.show()

Now we are going to find the power spectrum of the portion of the first sound before the beat with only one tuning fork playing. There are several ways to obtain a segment of the sound, including taking a "slice" of the `beats_wave` variable. The easiest way is just to read a segment of the soung because the wavefile.read function will pick out the times for you.

Read the segment of `Beats.wav` from 1.65 s to 2.68 s by entering the following cell.

In [0]:
onefork_info, onefork_wave = wavefile.read("Beats.wav", t0=1.65, t1=2.68)

Now take a power spectrum of this sound and plot it by entering the following cell. (Notice how we set the x-axis to only show frequencies from 0 Hz to 1000 Hz. You can still zoom out.)

In [0]:
nfft = onefork_info["fs"]
onefork_freq, onefork_ps = signal.welch(onefork_wave, onefork_info['fs'], nperseg=nfft)
fig = go.Figure(go.Scatter(x=onefork_freq, y=10*np.log10(onefork_ps), 
                                     line={'color': 'blue'}))
fig.update_xaxes(title_text='Frequency (Hz)', range=[0,1000])
fig.update_yaxes(title_text='Power Spectral Density (dB)', range=[-10, 80])
fig.show()

Now we will take the power spectrum of the portion of the recording from 2.8 s to the end. Read this segment from the file by entering the following cell.

In [0]:
twofork_info, twofork_wave = wavefile.read("Beats.wav", t0=2.8, t1=-1)

In [0]:
fig = go.Figure(go.Scatter(x=np.arange(twofork_info['Nsamples']) / twofork_info['fs'], y=twofork_wave, line={'color': 'blue'}))
fig.show()

Now we will use the same parameters to make a power spectrum of this sound.

In [0]:
nfft = twofork_info["fs"]
twofork_freq, twofork_ps = signal.welch(twofork_wave, twofork_info["fs"], nperseg=nfft)
fig = go.Figure(go.Scatter(x=twofork_freq, y=10*np.log10(twofork_ps), 
                                     line={'color': 'blue'}))
fig.update_xaxes(title_text='Frequency (Hz)', range=[0, 1000])
fig.update_yaxes(title_text='Power Spectral Density (dB)')
fig.show()

## Spectrogram

Another important tool for spectral analysis is the spectrogram. A spectrogram is a series of short-time power spectra, which shows how the frequency content changes in time. A spectrogram is a 3-D plot. One axis (usually the x-axis) is time. The second axis (usually the y-axis) is frequency. The z-axis (represented by color) is power spectral density, which represents how much signal power is in that frequency bin. (Note that sometimes but not often spectrograms have time on the y-axis and frequency on the x-axis.) Each slice of a spectrogram at a given time is a power spectrum.

Two important parameters for spectrograms are the number of FFT points. This is the number of samples used to calculate each power spectrum slice. Increasing the number of FFT samples increases the frequency resolution, but it decreases the time resolution because each power spectrum slice takes more time (more samples = more time). The second important parameter is the overlap between FFTs. This means the samples used for each FFT contain some of the samples from the previous FFT. The optimal overlap is half of the points used for each FFT.  In other words, it you use 2048 points for your FFT, the optimal overlap would be 1024 points. You can use an overlap than 1/2 of the FFT points to make smoother plots, but there is no increase of information for these higher overlaps.

In [0]:
nfft = 2048
noverlap = nfft/2

Another important parameter is the window function to use. We will use a Hanning window function named `signal.hann`. (This is actually the default window function used for the power spectrum, but we did not have to specify it in tht function call.)

Enter the cell below to calcualate the spectrogram using the number of FFT points and overlap we specified and the Hanning window function.

In [0]:
sg = signal.spectrogram(pipe_wave, fs=pipe_info['fs'], window=signal.hann(nfft), nperseg=nfft, 
        noverlap=noverlap)

In [0]:
fig = go.Figure(data=go.Heatmap(z=10*np.log10(sg[2]), x=sg[1], y=sg[0], colorscale='Viridis', 
                                  colorbar={'title': {'text': 'PSD (dB)', 'side': 'right'}}))
fig.update_xaxes(title_text='Time (s)')
fig.update_yaxes(title_text='Frequency (Hz)')
fig.show()

We can adjust some of the spectrogram parameters to show some of the sound characteristics more clearly. We can zoom and pan the x and y axes on a Plotly plot, but we cannot change the color scale provided when we produce the plot. In order to change the color scale, we must make another plot. In the example below we use the variables `psdmin` and `psdmax` to set the minimum and maximum power spectral density (PSD) values for the color scale. We will also use `fmin` and `fmax` for the minimum and maximum frequencies displayed in the graph. (You can zoom the frequency scale in or out on the graph, though.)

In [0]:
psdmin = 10
psdmax = 70
fmin = 0
fmax = 2000

The code in the cell below produces a spectrogram with the paramenters set above.

In [0]:
fig = go.Figure(data=go.Heatmap(z=10*np.log10(sg[2]), zmin=psdmin, zmax=psdmax, x=sg[1], y=sg[0], colorscale='Viridis', 
                                  colorbar={'title': {'text': 'PSD (dB)', 'side': 'right'}}))
fig.update_xaxes(title_text='Time (s)')
fig.update_yaxes(title_text='Frequency (Hz)', range=[fmin, fmax])
fig.show()

## Wave Plot and Spectrogram

Sometimes it is good to show both the waveform and the spectrogram of a sound in a combined graph with subplots. Plotly can produce a spectrogram with subplots with a shared x-axis. Zooming or panning the x-axis on one subplot zooms or pans the other subplot.

In [0]:
fig2 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
fig2.add_trace(go.Heatmap(z=10*np.log10(sg[2]), zmin=psdmin, zmax=psdmax, x=sg[1], y=sg[0], 
                      colorscale='Viridis', colorbar={'title': {'text': 'Power Spectral Density (dB)', 'side': 'right'}}), 
                     row=2, col=1)
fig2.add_trace(go.Scatter(x=pipe_times, y=pipe_wave, line={'color': 'blue'}), row=1, col=1)
fig2.update_xaxes(title_text='Time (s)')
fig2.update_yaxes(title_text='Relative Pressure', row=1, col=1)
fig2.update_yaxes(title_text='Frequency (Hz)', range=[fmin, fmax], row=2, col=1)